In [1]:
import numpy as np
import pandas as pd
import datetime
import re
from dateutil.parser import parse
import glob
import sys
import openpyxl as opx
import xlrd
import seaborn as sns
import random
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import font_manager, rc
matplotlib.rcParams['axes.unicode_minus'] = False
font_path = "C:/Windows/Fonts/malgun.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# KPEC 설비간 에너지흐름차트 로딩

In [2]:
df = pd.read_csv('./data/220728/220728 facility_energy_flow.csv', parse_dates = True)

In [3]:
df

,DateTime,PVout(We),PVTout(We),FCout(We),PlugLoad(We),LightLoad(We),FromGrid(We),ToGrid(We),ToESS(We),ESSout(We),...,GSHP2HTES(Wt),ASHP2Cool(Wt),ASHP2Heat(Wt),CTES2Cool(Wt),HTES2Heat(Wt),HTES2Water(Wt),DirectElec(We),CoolLoad(Wt),HeatLoad(Wt),HotwaterLoad(Wt)
0,2022-07-28 18:01:00,99.967,-71.783,0.0,7420.80,2384.15,13781.8,0.0,0.0,633.167,...,0.0,6000.86,0.0,11337.70,0.0,0.0,9804.95,17338.60,0.0,0.0
1,2022-07-28 18:00:00,149.533,-72.533,0.0,7060.10,2347.02,12846.5,0.0,0.0,999.550,...,0.0,5784.01,0.0,10499.10,0.0,0.0,9407.12,16283.10,0.0,0.0
2,2022-07-28 17:59:00,151.617,-65.300,0.0,7003.08,2397.72,12629.6,0.0,0.0,999.433,...,0.0,4850.01,0.0,9523.61,0.0,0.0,9400.80,14373.60,0.0,0.0
3,2022-07-28 17:58:00,184.050,-64.184,0.0,6934.88,2319.27,12194.6,0.0,0.0,999.417,...,0.0,3958.87,0.0,8698.12,0.0,0.0,9254.15,12657.00,0.0,0.0
4,2022-07-28 17:57:00,177.049,-65.234,0.0,6904.10,2252.63,12006.3,0.0,0.0,999.650,...,0.0,3809.71,0.0,9025.99,0.0,0.0,9156.73,12835.70,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425984,2021-09-13 00:17:00,-78.000,10.000,0.0,5304.00,1026.00,7718.0,0.0,0.0,0.000,...,0.0,0.00,0.0,2730.33,0.0,0.0,6330.00,2730.33,0.0,0.0
425985,2021-09-13 00:15:00,-82.000,10.000,0.0,5614.00,1060.00,8095.0,0.0,0.0,0.000,...,0.0,0.00,0.0,3222.91,0.0,0.0,6674.00,3222.91,0.0,0.0
425986,2021-09-13 00:11:00,-80.000,10.000,0.0,4922.00,1092.00,6261.0,0.0,0.0,0.000,...,0.0,0.00,0.0,0.00,0.0,0.0,6014.00,0.00,0.0,0.0
425987,2021-09-13 00:06:00,-78.000,10.000,0.0,4412.00,1043.00,5701.0,0.0,0.0,0.000,...,0.0,0.00,0.0,0.00,0.0,0.0,5455.00,0.00,0.0,0.0


In [4]:
df['DateTime']= pd.to_datetime(df['DateTime'])

In [5]:
df['GSHP_inputE(We)'] = df['Grid2GSHP(We)'] + df['ESS2GSHP(We)']

In [6]:
df_gshp = df[['DateTime', 'GSHP_inputE(We)', 'GSHP2CTES(Wt)', 'CTES2Cool(Wt)', 'GSHP2HTES(Wt)', 'HTES2Heat(Wt)']]

In [7]:
df_gshp = df_gshp.set_index('DateTime')

In [8]:
df_gshp_1d = df_gshp.resample('1d').sum()/60/1000

In [9]:
df_gshp_1d = df_gshp_1d.rename(columns={'GSHP_inputE(We)':'GSHP_inputE(kWh)',
                                        'GSHP2CTES(Wt)':'GSHP2CTES(kWh)',
                                        'CTES2Cool(Wt)':'CTESCool(kWh)',
                                        'GSHP2HTES(Wt)':'GSHP2HTES(kWh)',
                                        'HTES2Heat(Wt)':'HTES2Heat(kWh)'})

In [10]:
df_gshp_1d

,GSHP_inputE(kWh),GSHP2CTES(kWh),CTESCool(kWh),GSHP2HTES(kWh),HTES2Heat(kWh)
DateTime,,,,,
2021-09-13,6.400583,144.088926,11.375311,0.0,0.0
2021-09-14,5.195100,10.894056,9.396547,0.0,0.0
2021-09-15,4.607483,12.527208,8.509174,0.0,0.0
2021-09-16,2.214883,0.947467,6.911199,0.0,0.0
2021-09-17,0.137133,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...
2022-07-24,15.046542,50.159244,32.999518,0.0,0.0
2022-07-25,10.534723,34.080490,20.964751,0.0,0.0
2022-07-26,12.774818,40.389657,32.243459,0.0,0.0


In [11]:
df_gshp_1d['Cooling_COP'] = df_gshp_1d['GSHP2CTES(kWh)'] / df_gshp_1d['GSHP_inputE(kWh)']
df_gshp_1d['Heating_COP'] = df_gshp_1d['GSHP2HTES(kWh)'] / df_gshp_1d['GSHP_inputE(kWh)']
df_gshp_1d['Heating_COP_load'] = df_gshp_1d['HTES2Heat(kWh)'] / df_gshp_1d['GSHP_inputE(kWh)'] # COP값이상해서 HTES2Heat로 대체

In [23]:
df_gshp_1d.loc['2022-05-01':'2022-06-15']

,GSHP_inputE(kWh),GSHP2CTES(kWh),CTESCool(kWh),GSHP2HTES(kWh),HTES2Heat(kWh),Cooling_COP,Heating_COP,Heating_COP_load
DateTime,,,,,,,,
2022-05-01,7.825170,0.000000,0.000000,6.219125,17.007772,0.000000,0.794759,2.173470
2022-05-02,7.159207,0.000000,0.000000,6.011887,17.610562,0.000000,0.839742,2.459848
2022-05-03,12.588191,0.000000,0.000000,3.877403,55.888611,0.000000,0.308019,4.439765
2022-05-04,17.843405,0.000000,0.000000,4.646545,59.664799,0.000000,0.260407,3.343801
2022-05-05,17.775907,0.000000,0.000000,9.380220,43.774899,0.000000,0.527693,2.462597
2022-05-06,5.976173,0.000000,0.000000,1.796136,22.397072,0.000000,0.300550,3.747728
2022-05-07,5.235042,0.000000,0.000000,5.344897,21.314057,0.000000,1.020985,4.071421
2022-05-08,8.519345,0.000000,0.000000,2.608260,22.998497,0.000000,0.306157,2.699562
2022-05-09,13.085186,0.000000,0.000000,7.703372,34.528265,0.000000,0.588709,2.638729


In [13]:
df_gshp_1d.loc['2021-11-01':'2021-11-30']

,GSHP_inputE(kWh),GSHP2CTES(kWh),CTESCool(kWh),GSHP2HTES(kWh),HTES2Heat(kWh),Cooling_COP,Heating_COP,Heating_COP_load
DateTime,,,,,,,,
2021-11-01,20.265217,0.0,0.0,8.095153,155.201003,0.0,0.399460,7.658492
2021-11-02,8.136233,0.0,0.0,3.847731,45.541820,0.0,0.472913,5.597408
2021-11-03,8.565200,0.0,0.0,0.000000,11.344811,0.0,0.000000,1.324524
2021-11-04,1.235783,0.0,0.0,0.000000,109.273883,0.0,0.000000,88.424792
2021-11-05,0.359267,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
2021-11-06,0.000000,0.0,0.0,0.000000,0.000000,NaN,NaN,NaN
2021-11-07,0.000000,0.0,0.0,0.000000,0.000000,NaN,NaN,NaN
2021-11-08,0.000000,0.0,0.0,0.000000,0.000000,NaN,NaN,NaN
2021-11-09,4.360367,0.0,0.0,0.887740,26.206810,0.0,0.203593,6.010231


In [35]:
df_gshp_1d.loc['2022-07-01':'2022-07-31']

,GSHP_inputE(kWh),GSHP2CTES(kWh),CTESCool(kWh),GSHP2HTES(kWh),HTES2Heat(kWh),Cooling_COP,Heating_COP,Heating_COP_load
DateTime,,,,,,,,
2022-07-01,8.125561,7.377658,3.560550,0.0,0.0,0.907957,0.0,0.0
2022-07-02,0.853827,1.058259,4.218603,0.0,0.0,1.239430,0.0,0.0
2022-07-03,0.140759,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
2022-07-04,0.141559,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
2022-07-05,0.136330,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
2022-07-06,15.161244,17.608600,12.393586,0.0,0.0,1.161422,0.0,0.0
2022-07-07,14.079083,15.375731,9.353791,0.0,0.0,1.092097,0.0,0.0
2022-07-08,10.028101,10.285249,4.412897,0.0,0.0,1.025643,0.0,0.0
2022-07-09,15.180546,15.748819,16.381954,0.0,0.0,1.037434,0.0,0.0


# 1시간단위로 분석

In [ ]:
df_gshp_1h = df_gshp.resample('1h').sum()/60/1000

In [ ]:
df_gshp_1h = df_gshp_1h.rename(columns={'GSHP_inputE(We)':'GSHP_inputE(kWh)',
                                        'GSHP2CTES(Wt)':'GSHP2CTES(kWh)',
                                        'CTES2Cool(Wt)':'CTESCool(kWh)',
                                        'GSHP2HTES(Wt)':'GSHP2HTES(kWh)',
                                        'HTES2Heat(Wt)':'HTES2Heat(kWh)'})

In [ ]:
df_gshp_1h

In [ ]:
df_gshp_1h['Cooling_COP'] = df_gshp_1h['GSHP2CTES(kWh)'] / df_gshp_1h['GSHP_inputE(kWh)']
df_gshp_1h['Heating_COP'] = df_gshp_1h['GSHP2HTES(kWh)'] / df_gshp_1h['GSHP_inputE(kWh)']
df_gshp_1h['Heating_COP_load'] = df_gshp_1h['HTES2Heat(kWh)'] / df_gshp_1h['GSHP_inputE(kWh)']

In [ ]:
df_gshp_1h.loc['2022-07-27':'2022-07-28']

# 1달단위로 분석

In [14]:
df_gshp_1M = df_gshp.resample('1M').sum()/60/1000

In [15]:
df_gshp_1M = df_gshp_1M.rename(columns={'GSHP_inputE(We)':'GSHP_inputE(kWh)',
                                        'GSHP2CTES(Wt)':'GSHP2CTES(kWh)',
                                        'CTES2Cool(Wt)':'CTESCool(kWh)',
                                        'GSHP2HTES(Wt)':'GSHP2HTES(kWh)',
                                        'HTES2Heat(Wt)':'HTES2Heat(kWh)'})

In [18]:
df_gshp_1M['Cooling_COP'] = df_gshp_1M['GSHP2CTES(kWh)'] / df_gshp_1M['GSHP_inputE(kWh)']
df_gshp_1M['Heating_COP'] = df_gshp_1M['GSHP2HTES(kWh)'] / df_gshp_1M['GSHP_inputE(kWh)']
df_gshp_1M['Heating_COP_load'] = df_gshp_1M['HTES2Heat(kWh)'] / df_gshp_1M['GSHP_inputE(kWh)']

In [19]:
df_gshp_1M

,GSHP_inputE(kWh),GSHP2CTES(kWh),CTESCool(kWh),GSHP2HTES(kWh),HTES2Heat(kWh),Cooling_COP,Heating_COP,Heating_COP_load
DateTime,,,,,,,,
2021-09-30,106.950650,332.250541,228.201902,0.000000,0.000000,3.106578,0.000000,0.000000
2021-10-31,331.395117,227.893435,143.256806,44.165752,2321.640319,0.687679,0.133272,7.005656
2021-11-30,451.967545,0.000000,0.000000,136.840764,3939.025347,0.000000,0.302767,8.715284
2021-12-31,844.503059,0.000000,0.000000,341.880485,2704.086396,0.000000,0.404830,3.201985
2022-01-31,1146.721713,0.000000,0.000000,459.999598,3825.763251,0.000000,0.401143,3.336261
2022-02-28,966.167974,0.000000,0.000000,380.503791,3253.456037,0.000000,0.393828,3.367381
2022-03-31,480.702286,0.000000,0.000000,206.500065,1616.837031,0.000000,0.429580,3.363489
2022-04-30,185.741038,0.000000,0.000000,114.111025,533.865854,0.000000,0.614355,2.874248
2022-05-31,271.979197,245.610450,728.338559,91.283061,554.222982,0.903049,0.335625,2.037740


In [ ]:
df3=df3.set_index('DateTime')

In [ ]:
df3

# KPEC 건물간 에너지흐름차트 로딩

In [ ]:
df4 = pd.read_csv('./data/220114/220114 building_information.csv', parse_dates = True)

In [ ]:
df4['DateTime']= pd.to_datetime(df4['DateTime'])

In [ ]:
df4=df4[(df4['DateTime']<'2022-01-01 00:00:00')]

In [ ]:
df4=df4.set_index('DateTime')

# KPEC 데이터 합치기

In [ ]:
df_kpec = pd.DataFrame()
df_kpec = pd.DataFrame(pd.date_range('2021-09-13 00:00:00', '2022-02-28 23:59:00', freq='T'), columns=['Time'])

In [ ]:
df_kpec

In [ ]:
df_kpec = df_kpec.set_index('Time')

In [ ]:
df_kpec['Cons_kpec(W)'] = df1['energyTotalConsume']
df_kpec['Prod_kpec(W)'] = df1['energyTotalProduce']
df_kpec['ESS_kpec(W)'] = df2['ESS1_Power(W)']*(-1)

In [ ]:
df_kpec

In [ ]:
df_kpec.info()

In [ ]:
df_kpec['PV_kpec(We)'] = df3['PVout(We)']
df_kpec['PVT_kpec(We)'] = df3['PVTout(We)']
df_kpec['FC_kpec(We)'] = df3['FCout(We)']
df_kpec['PlugLoad(We)'] = df3['PlugLoad(We)']
df_kpec['LightLoad(We)'] = df3['LightLoad(We)']
df_kpec['FromGrid(We)'] = df3['FromGrid(We)']
df_kpec['ToGrid(We)'] = df3['ToGrid(We)']
df_kpec['ToESS(We)'] = df3['ToESS(We)']
df_kpec['ESSout(We)'] = df3['ESSout(We)']
df_kpec['ESS2Grid(We)'] = df3['ESS2Grid(We)']
df_kpec['ESS2ASHP(We)'] = df3['ESS2ASHP(We)']
df_kpec['ESS2GSHP(We)'] = df3['ESS2GSHP(We)']
df_kpec['Grid2ASHP(We)'] = df3['Grid2ASHP(We)']
df_kpec['Grid2GSHP(We)'] = df3['Grid2GSHP(We)']
df_kpec['PVTHout(Wt)'] = df3['PVTHout(Wt)']
df_kpec['FCHout(Wt)'] = df3['FCHout(Wt)']
df_kpec['ASHP2HTES(Wt)'] = df3['ASHP2HTES(Wt)']
df_kpec['GSHP2CTES(Wt)'] = df3['GSHP2CTES(Wt)']
df_kpec['GSHP2HTES(Wt)'] = df3['GSHP2HTES(Wt)']
df_kpec['ASHP2Cool(Wt)'] = df3['ASHP2Cool(Wt)']
df_kpec['ASHP2Heat(Wt)'] = df3['ASHP2Heat(Wt)']
df_kpec['CTES2Cool(Wt)'] = df3['CTES2Cool(Wt)']
df_kpec['HTES2Heat(Wt)'] = df3['HTES2Heat(Wt)']
df_kpec['HTES2Water(Wt)'] = df3['HTES2Water(Wt)']
df_kpec['DirectElec(We)'] = df3['DirectElec(We)']
df_kpec['CoolLoad(Wt)'] = df3['CoolLoad(Wt)']
df_kpec['HeatLoad(Wt)'] = df3['HeatLoad(Wt)']
df_kpec['HotwaterLoad(Wt)'] = df3['HotwaterLoad(Wt)']

In [ ]:
df_kpec['PV_kpec(We)'] = np.where(df_kpec['PV_kpec(We)']<0, 0, df_kpec['PV_kpec(We)'])
df_kpec['PVT_kpec(We)'] = np.where(df_kpec['PVT_kpec(We)']<0, 0, df_kpec['PVT_kpec(We)'])
df_kpec['FC_kpec(We)'] = np.where(df_kpec['FC_kpec(We)']<0, 0, df_kpec['FC_kpec(We)'])
df_kpec['PlugLoad(We)'] = np.where(df_kpec['PlugLoad(We)']<0, 0, df_kpec['PlugLoad(We)'])

In [ ]:
df_kpec['Cons_acc_kpec(Wh)'] = df_kpec['Cons_kpec(W)'].cumsum()*60*1/3600
df_kpec['Prod_acc_kpec(Wh)'] = df_kpec['Prod_kpec(W)'].cumsum()*60*1/3600
df_kpec['ESS_acc_kpec(Wh)'] = df_kpec['ESS_kpec(W)'].cumsum()*60*1/3600

df_kpec['PV_acc_kpec(Wh)'] = df_kpec['PV_kpec(We)'].cumsum()*60*1/3600
df_kpec['PVT_acc_kpec(Wh)'] = df_kpec['PVT_kpec(We)'].cumsum()*60*1/3600
df_kpec['FC_acc_kpec(Wh)'] = df_kpec['FC_kpec(We)'].cumsum()*60*1/3600
df_kpec['PlugLoad_acc(Wh)'] = df_kpec['PlugLoad(We)'].cumsum()*60*1/3600
df_kpec['LightLoad_acc(Wh)'] = df_kpec['LightLoad(We)'].cumsum()*60*1/3600
df_kpec['FromGrid_acc(Wh)'] = df_kpec['FromGrid(We)'].cumsum()*60*1/3600
df_kpec['ToGrid_acc(Wh)'] = df_kpec['ToGrid(We)'].cumsum()*60*1/3600
df_kpec['ToESS_acc(Wh)'] = df_kpec['ToESS(We)'].cumsum()*60*1/3600
df_kpec['ESSout_acc(Wh)'] = df_kpec['ESSout(We)'].cumsum()*60*1/3600
df_kpec['ESS2Grid_acc(Wh)'] = df_kpec['ESS2Grid(We)'].cumsum()*60*1/3600
df_kpec['ESS2ASHP_acc(Wh)'] = df_kpec['ESS2ASHP(We)'].cumsum()*60*1/3600
df_kpec['ESS2GSHP_acc(Wh)'] = df_kpec['ESS2GSHP(We)'].cumsum()*60*1/3600
df_kpec['Grid2ASHP_acc(Wh)'] = df_kpec['Grid2ASHP(We)'].cumsum()*60*1/3600
df_kpec['Grid2GSHP_acc(Wh)'] = df_kpec['Grid2GSHP(We)'].cumsum()*60*1/3600
df_kpec['PVTHout_acc(Wh)'] = df_kpec['PVTHout(Wt)'].cumsum()*60*1/3600
df_kpec['FCHout_acc(Wh)'] = df_kpec['FCHout(Wt)'].cumsum()*60*1/3600
df_kpec['ASHP2HTES_acc(Wh)'] = df_kpec['ASHP2HTES(Wt)'].cumsum()*60*1/3600
df_kpec['GSHP2CTES_acc(Wh)'] = df_kpec['GSHP2CTES(Wt)'].cumsum()*60*1/3600
df_kpec['GSHP2HTES_acc(Wh)'] = df_kpec['GSHP2HTES(Wt)'].cumsum()*60*1/3600
df_kpec['ASHP2Cool_acc(Wh)'] = df_kpec['ASHP2Cool(Wt)'].cumsum()*60*1/3600
df_kpec['ASHP2Heat_acc(Wh)'] = df_kpec['ASHP2Heat(Wt)'].cumsum()*60*1/3600
df_kpec['CTES2Cool_acc(Wh)'] = df_kpec['CTES2Cool(Wt)'].cumsum()*60*1/3600
df_kpec['HTES2Heat_acc(Wh)'] = df_kpec['HTES2Heat(Wt)'].cumsum()*60*1/3600
df_kpec['HTES2Water_acc(Wh)'] = df_kpec['HTES2Water(Wt)'].cumsum()*60*1/3600
df_kpec['DirectElec_acc(Wh)'] = df_kpec['DirectElec(We)'].cumsum()*60*1/3600
df_kpec['CoolLoad_acc(Wh)'] = df_kpec['CoolLoad(Wt)'].cumsum()*60*1/3600
df_kpec['HeatLoad_acc(Wh)'] = df_kpec['HeatLoad(Wt)'].cumsum()*60*1/3600
df_kpec['HotwaterLoad_acc(Wh)'] = df_kpec['HotwaterLoad(Wt)'].cumsum()*60*1/3600

In [ ]:
df_kpec['E_self(W)'] = df_kpec['Prod_kpec(W)'] - df_kpec['ESS_kpec(W)'] - df_kpec['ToGrid(We)']
df_kpec['E_self_acc(Wh)'] =df_kpec['E_self(W)'].cumsum()*60*1/3600

In [ ]:
df_kpec['denominator'] = df_kpec.loc[:,['E_self(W)','Cons_kpec(W)']].min(axis=1)
df_kpec['denominator_acc'] = df_kpec.loc[:,['E_self_acc(Wh)','Cons_acc_kpec(Wh)']].min(axis=1)

In [ ]:
df_kpec['SCF'] = df_kpec['denominator']/(df_kpec['Prod_kpec(W)'] - df_kpec['ESS_kpec(W)'])
df_kpec['LCF'] = df_kpec['denominator']/df_kpec['Cons_kpec(W)']

In [ ]:
df_kpec['SCF_acc'] = df_kpec['denominator_acc']/(df_kpec['Prod_acc_kpec(Wh)'] - df_kpec['ESS_acc_kpec(Wh)'])
df_kpec['LCF_acc'] = df_kpec['denominator_acc']/df_kpec['Cons_acc_kpec(Wh)']

In [ ]:
df_kpec['self_sustain'] = df_kpec['Prod_acc_kpec(Wh)']/df_kpec['Cons_acc_kpec(Wh)']
df_kpec['Avg_SCF&LCF']=(df_kpec['SCF_acc']+df_kpec['LCF_acc'])/2

In [ ]:
df_kpec.info()

In [ ]:
df_kpec

In [ ]:
df_kpec.to_csv('./data/220302_kpec.csv')

In [ ]:
fig, ax = plt.subplots(figsize=(18,300))

for i in range(len(df_kpec.columns)):
    plt.subplot(len(df_kpec.columns), 1, i+1)
    plt.subplots_adjust(hspace=0.4)
    name = df_kpec.columns[i]
    r=random.random()
    g=random.random()
    b=random.random()
    color=(r,g,b)
    plt.plot(df_kpec[name], color=color)    
    plt.title(name, loc = 'left', pad=15, fontsize=13, fontweight="bold")      
    plt.yticks(fontsize=13)    
    plt.xticks(fontsize=13)                
    plt.xlim(pd.Timestamp('2021-09-07'), pd.Timestamp('2022-03-07'))    
    plt.grid(True)
plt.show()
fig.tight_layout()

# 주거용 건물

## KePSH1

In [ ]:
data = pd.read_csv('./data/220114/220114 kepsh1.csv', parse_dates = True)

In [ ]:
all_features = data.columns.tolist()
print(all_features)

In [ ]:
df_kepsh1 = pd.DataFrame()
df_kepsh1['Time'] = data['DateTime']
df_kepsh1['PV_kepsh1(W)'] = np.where(data['Pv_Supply(W)']<0, 0, data['Pv_Supply(W)'])
df_kepsh1['PVT_kepsh1(W)'] = np.where(data['Pvt_Supply(W)']<0, 0, data['Pvt_Supply(W)'])
df_kepsh1['FC(W)'] = data['FuelCell_Supply(W)']
df_kepsh1['HP_kepsh1(W)'] = data['Heatpump_Load(W)']
df_kepsh1['Cons_kepsh1(W)'] = data['Total_Consumption(W)']
df_kepsh1['Prod_kepsh1(W)'] = data['Total_Production(W)']
df_kepsh1['ESS(W)'] = data['ESS_Power(W)']*(-1)

df_kepsh1.head()


In [ ]:
df_kepsh1['Time'] = pd.to_datetime(df_kepsh1['Time'])

In [ ]:
df_kepsh1 = df_kepsh1[(df_kepsh1['Time']>='2021-09-13 00:00:00') ]
df_kepsh1 = df_kepsh1[(df_kepsh1['Time']<='2021-12-31 23:59:00') ]

In [ ]:
df_kepsh1 = df_kepsh1.set_index('Time')

In [ ]:
df_kepsh1.info()

In [ ]:
df_kepsh1['Cons_kepsh1(W)'] = df_kepsh1['Cons_kepsh1(W)'].astype(float)
df_kepsh1['Prod_kepsh1(W)'] = df_kepsh1['Prod_kepsh1(W)'].astype(float)

In [ ]:
df_kepsh1['Cons_acc_kepsh1(Wh)'] = df_kepsh1['Cons_kepsh1(W)'].cumsum()*60/3600
df_kepsh1['Prod_acc_kepsh1(Wh)'] = df_kepsh1['Prod_kepsh1(W)'].cumsum()*60/3600

In [ ]:
df_kepsh1['PV_acc_kepsh1(Wh)'] = df_kepsh1['PV_kepsh1(W)'].cumsum()*60/3600
df_kepsh1['PVT_acc_kepsh1(Wh)'] = df_kepsh1['PVT_kepsh1(W)'].cumsum()*60/3600
df_kepsh1['FC_acc(Wh)'] = df_kepsh1['FC(W)'].cumsum()*60/3600

In [ ]:
df_kepsh1['denominator'] = df_kepsh1.loc[:,['Cons_kepsh1(W)','Prod_kepsh1(W)']].min(axis=1)
df_kepsh1['denominator_acc'] = df_kepsh1.loc[:,['Cons_acc_kepsh1(Wh)','Prod_acc_kepsh1(Wh)']].min(axis=1)

In [ ]:
df_kepsh1['SCF'] = df_kepsh1['denominator']/df_kepsh1['Prod_kepsh1(W)']
df_kepsh1['LCF'] = df_kepsh1['denominator']/df_kepsh1['Cons_kepsh1(W)']

In [ ]:
df_kepsh1['SCF_acc'] = df_kepsh1['denominator_acc']/df_kepsh1['Prod_acc_kepsh1(Wh)']
df_kepsh1['LCF_acc'] = df_kepsh1['denominator_acc']/df_kepsh1['Cons_acc_kepsh1(Wh)']
df_kepsh1['self_sustain'] = df_kepsh1['Prod_acc_kepsh1(Wh)']/df_kepsh1['Cons_acc_kepsh1(Wh)']
df_kepsh1['Avg_SCF&LCF'] = (df_kepsh1['SCF_acc']+df_kepsh1['LCF_acc'])/2

In [ ]:
df_kepsh1

In [ ]:
df_kepsh1.to_csv('./data/220114_kepsh1.csv')

In [ ]:
fig, ax = plt.subplots(figsize=(18,70))

for i in range(len(df_kepsh1.columns)):
    plt.subplot(len(df_kepsh1.columns), 1, i+1)
    plt.subplots_adjust(hspace=0.5)
    name = df_kepsh1.columns[i]
    r=random.random()
    g=random.random()
    b=random.random()
    color=(r,g,b)
    plt.plot(df_kepsh1[name], color=color)    
    plt.title(name, loc = 'left', pad=15, fontsize=13, fontweight="bold")      
    plt.yticks(fontsize=13)    
    plt.xticks(fontsize=13)                
    plt.xlim(pd.Timestamp('2021-09-07'), pd.Timestamp('2022-01-07'))    
    plt.grid(True)
plt.show()
fig.tight_layout()

## KePSH2

In [ ]:
data = pd.read_csv('./data/220114/220114 kepsh2.csv', parse_dates = True)

In [ ]:
data

In [ ]:
df_kepsh2 = pd.DataFrame()
df_kepsh2['Time'] = data['DateTime']
# PV, PVT 음수값은 0으로 처리
df_kepsh2['PV_kepsh2(W)'] = np.where(data['BIPV_Supply(W)']<0, 0, data['BIPV_Supply(W)'])
df_kepsh2['PVT_kepsh2(W)'] = np.where(data['Pvt_Supply(W)']<0, 0, data['Pvt_Supply(W)'])
df_kepsh2['Cons_kepsh2(W)'] = data['Total_Consumption(W)']
df_kepsh2['Prod_kepsh2(W)'] = data['Total_Production(W)']

df_kepsh2.head()


In [ ]:
df_kepsh2['Time'] = pd.to_datetime(df_kepsh2['Time'])

In [ ]:
df_kepsh2 = df_kepsh2[(df_kepsh2['Time']>='2021-09-13 00:00:00') ]
df_kepsh2 = df_kepsh2[(df_kepsh2['Time']<='2021-12-31 23:59:00') ]

In [ ]:
df_kepsh2['Cons_kepsh2(W)'] = df_kepsh2['Cons_kepsh2(W)'].astype(float)
df_kepsh2['Prod_kepsh2(W)'] = df_kepsh2['Prod_kepsh2(W)'].astype(float)

In [ ]:
df_kepsh2 = df_kepsh2.set_index('Time')

In [ ]:
df_kepsh2['Cons_acc_kepsh2(Wh)'] = df_kepsh2['Cons_kepsh2(W)'].cumsum()*60/3600
df_kepsh2['Prod_acc_kepsh2(Wh)'] = df_kepsh2['Prod_kepsh2(W)'].cumsum()*60/3600

In [ ]:
df_kepsh2['PV_acc_kepsh2(Wh)'] = df_kepsh2['PV_kepsh2(W)'].cumsum()*60/3600
df_kepsh2['PVT_acc_kepsh2(Wh)'] = df_kepsh2['PVT_kepsh2(W)'].cumsum()*60/3600

In [ ]:
df_kepsh2['denominator'] = df_kepsh2.loc[:,['Cons_kepsh2(W)','Prod_kepsh2(W)']].min(axis=1)
df_kepsh2['denominator_acc'] = df_kepsh2.loc[:,['Cons_acc_kepsh2(Wh)','Prod_acc_kepsh2(Wh)']].min(axis=1)

In [ ]:
df_kepsh2['SCF'] = df_kepsh2['denominator']/df_kepsh2['Prod_kepsh2(W)']
df_kepsh2['LCF'] = df_kepsh2['denominator']/df_kepsh2['Cons_kepsh2(W)']

In [ ]:
df_kepsh2['SCF_acc'] = df_kepsh2['denominator_acc']/df_kepsh2['Prod_acc_kepsh2(Wh)']
df_kepsh2['LCF_acc'] = df_kepsh2['denominator_acc']/df_kepsh2['Cons_acc_kepsh2(Wh)']
df_kepsh2['self_sustain'] = df_kepsh2['Prod_acc_kepsh2(Wh)']/df_kepsh2['Cons_acc_kepsh2(Wh)']
df_kepsh2['Avg_SCF&LCF'] = (df_kepsh2['SCF_acc']+df_kepsh2['LCF_acc'])/2

In [ ]:
df_kepsh2.info()

In [ ]:
df_kepsh2

In [ ]:
fig, ax = plt.subplots(figsize=(18,60))

for i in range(len(df_kepsh2.columns)):
    plt.subplot(len(df_kepsh2.columns), 1, i+1)
    plt.subplots_adjust(hspace=0.5)
    name = df_kepsh2.columns[i]
    r=random.random()
    g=random.random()
    b=random.random()
    color=(r,g,b)
    plt.plot(df_kepsh2[name], color=color)    
    plt.title(name, loc = 'left', pad=15, fontsize=13, fontweight="bold")      
    plt.yticks(fontsize=13)    
    plt.xticks(fontsize=13)                
    plt.xlim(pd.Timestamp('2021-09-07'), pd.Timestamp('2022-01-07'))    
    plt.grid(True)
plt.show()
fig.tight_layout()